# Faster R-CNN

## Instala dependências e define variáveis globais

In [1]:
!pip install -q albumentations==1.4.6 opencv-python pycocotools tqdm
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q pycocotools albumentations==1.4.6 opencv-python tqdm
!pip uninstall -y sympy
!pip install sympy --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.5/153.5 kB 8.4 MB/s eta 0:00:00
Found existing installation: sympy 1.13.3
Uninstalling sympy-1.13.3:
  Successfully uninstalled sympy-1.13.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 83.9 MB/s eta 0:00:00


Define variáveis globais iniciais

In [2]:
import sys
sys.path.append("/content/drive/MyDrive/TCC/src/")

In [3]:
from pathlib import Path

DATASET_ROOT = Path("/content/drive/MyDrive/TCC/DATASET")

paths = {
    "bh_pools": {
        "images": {
            "train": DATASET_ROOT/"bh_pools/images/train",
            "val":   DATASET_ROOT/"bh_pools/images/val",
            "test":  DATASET_ROOT/"bh_pools/images/test",
        },
        "ann": {
            "train": DATASET_ROOT/"bh_pools/annotations/instances_train.json",
            "val":   DATASET_ROOT/"bh_pools/annotations/instances_val.json",
            "test":  DATASET_ROOT/"bh_pools/annotations/instances_test.json",
        }
    },
    "bh_watertanks": {
        "images": {
            "train": DATASET_ROOT/"bh_watertanks/images/train",
            "val":   DATASET_ROOT/"bh_watertanks/images/val",
            "test":  DATASET_ROOT/"bh_watertanks/images/test",
        },
        "ann": {
            "train": DATASET_ROOT/"bh_watertanks/annotations/instances_train.json",
            "val":   DATASET_ROOT/"bh_watertanks/annotations/instances_val.json",
            "test":  DATASET_ROOT/"bh_watertanks/annotations/instances_test.json",
        }
    }
}

## Define pesos e instancia model

In [4]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_classes = 1 + 2  # background + (piscina, caixa d'água) → ajuste se usar por dataset

weights = torchvision.models.detection.FasterRCNN_ResNet50_FPN_V2_Weights.COCO_V1
model = torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(weights=weights)

in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
model.to(device)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth


100%|██████████| 167M/167M [00:00<00:00, 197MB/s]


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       

## Define transformers e collate_fn


In [5]:
import json, cv2, numpy as np, torch, albumentations as A

def get_transforms(train=True, img_size=1024):
    if train:
        tfms = [
            A.LongestMaxSize(max_size=img_size),
            A.PadIfNeeded(img_size, img_size, border_mode=cv2.BORDER_CONSTANT, value=(114,114,114)),
            A.HorizontalFlip(p=0.5),
            A.ShiftScaleRotate(shift_limit=0.02, scale_limit=0.10, rotate_limit=10,
                               border_mode=cv2.BORDER_CONSTANT, value=(114,114,114), p=0.5),
            A.RandomBrightnessContrast(p=0.3),
            A.CLAHE(p=0.2),
        ]
    else:
        # tfms = [
        #     A.LongestMaxSize(max_size=img_size),
        #     A.PadIfNeeded(img_size, img_size, border_mode=cv2.BORDER_CONSTANT, value=(114,114,114)),
        # ]
        return None
    return A.Compose(
        tfms,
        bbox_params=A.BboxParams(
            format="pascal_voc",
            label_fields=["class_labels"],
            min_visibility=0.0,
        )
    )
def collate_fn(batch):
    imgs, targets = list(zip(*batch))
    return list(imgs), list(targets)

## Carrega dataloaders por classe

In [6]:
from models.coco_dataset import CocoDetDataset
from torch.utils.data import Dataset, DataLoader


# ----- BH-POOLS -----
pools_train = CocoDetDataset(
    paths["bh_pools"]["images"]["train"],
    paths["bh_pools"]["ann"]["train"],
    transforms=get_transforms(train=True))

pools_val   = CocoDetDataset(
    paths["bh_pools"]["images"]["val"],
    paths["bh_pools"]["ann"]["val"],
    transforms=get_transforms(train=False))

pools_test  = CocoDetDataset(
    paths["bh_pools"]["images"]["test"],
    paths["bh_pools"]["ann"]["test"],
    transforms=get_transforms(train=False))


pools_train_loader = DataLoader(pools_train, batch_size=2, shuffle=True,  num_workers=2, collate_fn=collate_fn, pin_memory=True)
pools_val_loader   = DataLoader(pools_val,   batch_size=2, shuffle=False, num_workers=2, collate_fn=collate_fn, pin_memory=True)
pools_test_loader  = DataLoader(pools_test,  batch_size=2, shuffle=False, num_workers=2, collate_fn=collate_fn, pin_memory=True)

print("BH-POOLS:", len(pools_train), len(pools_val), len(pools_test))

# ----- BH-WATERTANKS -----
wt_train = CocoDetDataset(
    paths["bh_watertanks"]["images"]["train"],
    paths["bh_watertanks"]["ann"]["train"],
    transforms=get_transforms(train=True),
    class_id_out=2)

wt_val   = CocoDetDataset(
    paths["bh_watertanks"]["images"]["val"],
    paths["bh_watertanks"]["ann"]["val"],
    transforms=get_transforms(train=False),
    class_id_out=2)

wt_test  = CocoDetDataset(
    paths["bh_watertanks"]["images"]["test"],
    paths["bh_watertanks"]["ann"]["test"],
    transforms=get_transforms(train=False),
    class_id_out=2)

wt_train_loader = DataLoader(wt_train, batch_size=2, shuffle=True,  num_workers=2, collate_fn=collate_fn, pin_memory=True)
wt_val_loader   = DataLoader(wt_val,   batch_size=2, shuffle=False, num_workers=2, collate_fn=collate_fn, pin_memory=True)
wt_test_loader  = DataLoader(wt_test,  batch_size=2, shuffle=False, num_workers=2, collate_fn=collate_fn, pin_memory=True)

print("BH-WATERTANKS:", len(wt_train), len(wt_val), len(wt_test))


loading annotations into memory...
Done (t=1.84s)
creating index...
index created!
loading annotations into memory...
Done (t=0.56s)
creating index...
index created!
loading annotations into memory...
Done (t=0.70s)
creating index...
index created!
BH-POOLS: 125 25 50
loading annotations into memory...
Done (t=1.56s)
creating index...
index created!
loading annotations into memory...
Done (t=0.60s)
creating index...
index created!
loading annotations into memory...
Done (t=1.04s)
creating index...
index created!
BH-WATERTANKS: 99 25 24


## Instancia classe COCO com datasets combinados

Combina datasets de piscinas e caixas dagua, e instancia loader misto

In [7]:
from torch.utils.data import ConcatDataset

train_ds = ConcatDataset([pools_train, wt_train])
val_ds   = ConcatDataset([pools_val, wt_val])
test_ds  = ConcatDataset([pools_test, wt_test])

In [8]:
BATCH_SIZE = 2

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=2, collate_fn=collate_fn, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=2, collate_fn=collate_fn, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=2, collate_fn=collate_fn, pin_memory=True)

## Instância funções de treinamento (fit, train_one_epoch e eval_val_loss)

In [9]:
from tqdm import tqdm
import math, os
from pycocotools.cocoeval import COCOeval

def train_one_epoch(model, loader, optimizer, scaler, desc="train"):
    model.train()
    epoch_loss = 0.0
    for imgs, targets in tqdm(loader, desc=desc, leave=False):
        imgs = [im.to(device) for im in imgs]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast():
            loss_dict = model(imgs, targets)
            loss = sum(loss_dict.values())
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        epoch_loss += loss.item()
    return epoch_loss / max(1, len(loader))

@torch.no_grad()
def eval_val_loss(model, loader, desc="val_loss"):
    # Para calcular loss, o modelo precisa estar em .train() nos detectors do torchvision
    model.train()
    epoch_loss = 0.0
    for imgs, targets in tqdm(loader, desc=desc, leave=False):
        imgs = [im.to(device) for im in imgs]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict = model(imgs, targets)
        loss = sum(loss_dict.values())
        epoch_loss += loss.item()
    return epoch_loss / max(1, len(loader))

def fit(model, train_loader, val_loader, optimizer, scaler, scheduler, epochs=20, ckpt_path="fasterrcnn_best.pth"):
    best_val = float("inf")
    for epoch in range(1, epochs+1):
        train_loss = train_one_epoch(model, train_loader, optimizer, scaler, desc=f"train[{epoch}/{epochs}]")
        val_loss   = eval_val_loss(model,  val_loader,  desc=f"val  [{epoch}/{epochs}]")
        scheduler.step()

        coco_eval = evaluate(model, val_loader, device=device)
        map50 = coco_eval.coco_eval['bbox'].stats[1]  # AP@0.50

        # descongela backbone após 2-3 épocas
        if epoch == 3:
            for p in model.backbone.body.parameters():
                p.requires_grad = True
            # recomputa params do otimizador (agora com backbone liberado)
            new_params = [p for p in model.parameters() if p.requires_grad]
            # LR menor após descongelar ajuda
            optimizer = torch.optim.SGD(new_params, lr=0.002, momentum=0.9, weight_decay=5e-4)
            print('🔓 Backbone descongelado.')

        print(f"[{epoch:02d}] train_loss={train_loss:.4f} | val_loss={val_loss:.4f}")

        if val_loss < best_val:
            best_val = val_loss
            torch.save(model.state_dict(), ckpt_path)
            print(f"  ↳ ✅ melhor até agora (salvo em {ckpt_path})")


## Inicia treinamento

Inicia primeira rodada de treinamento com lr=0.005, momenttum=0.9 e decaimento de 5^(e^-4)

In [15]:
import numpy as np, torch
from pycocotools.cocoeval import COCOeval

def _xyxy_to_xywh(b):
    x1,y1,x2,y2 = b
    return [float(x1), float(y1), float(x2-x1), float(y2-y1)]

@torch.no_grad()
def eval_coco_shim(model, data_loader, iou_type="bbox", score_thr=0.0, max_dets=100):
    device = next(model.parameters()).device
    model.eval()
    ds = data_loader.dataset
    cocoGt = ds.coco  # precisa existir (seu dataset baseado em COCO tem .coco)

    preds = []
    for images, targets in data_loader:
        img_ids = [int(t["image_id"]) for t in targets]
        # blindagem: float32 [0,1]
        images  = [(im.float()/255.0 if im.dtype!=torch.float32 else im).to(device) for im in images]
        outputs = model(images)

        for img_id, out in zip(img_ids, outputs):
            boxes  = out["boxes"].detach().cpu().numpy()
            scores = out["scores"].detach().cpu().numpy()
            labels = out["labels"].detach().cpu().numpy()  # seus labels contíguos (1..K) ou 1 (single-class)

            if score_thr > 0:
                keep = scores >= score_thr
                boxes, scores, labels = boxes[keep], scores[keep], labels[keep]
            if max_dets is not None and len(scores) > max_dets:
                order = np.argsort(-scores)[:max_dets]
                boxes, scores, labels = boxes[order], scores[order], labels[order]

            for b, s, c in zip(boxes, scores, labels):
                preds.append({
                    "image_id": img_id,
                    "category_id": int(c),      # IMPORTANT: deve bater com ids do seu JSON (contíguos ou originais)
                    "bbox": _xyxy_to_xywh(b.tolist()),
                    "score": float(s)
                })

    if len(preds) == 0:
        # nada previsto -> devolve zeros
        return np.zeros(12, dtype=float), None

    cocoDt = cocoGt.loadRes(preds)
    ev = COCOeval(cocoGt, cocoDt, iou_type)
    ev.evaluate(); ev.accumulate(); ev.summarize()
    return ev.stats, ev


In [16]:
def coco_ap50_per_class(cocoEval):
    import numpy as np
    if cocoEval is None:
        return []
    precisions = cocoEval.eval['precision']  # [TxRxKxAxM]
    i = np.where(np.isclose(cocoEval.params.iouThrs, 0.5))[0][0]
    pr = precisions[i, :, :, 0, -1]          # [R, K]
    aps = np.nanmean(pr, axis=0)             # [K]
    cat_ids = cocoEval.params.catIds
    id_to_name = {c['id']: c['name'] for c in cocoEval.cocoGt.loadCats(cat_ids)}
    return [(cid, id_to_name.get(cid, str(cid)), float(ap) if np.isfinite(ap) else 0.0)
            for cid, ap in zip(cat_ids, aps)]


In [17]:
def fit_map(model, train_loader, val_loader, optimizer, scaler, scheduler,
            epochs=10, ckpt_path="best.pth", score_thr_eval=0.0):
    best_ap50 = -1.0
    for epoch in range(1, epochs+1):
        # ===== treino =====
        model.train()
        running = 0.0
        for imgs, targets in train_loader:
            imgs = [(im.float()/255.0 if im.dtype!=torch.float32 else im).to(device) for im in imgs]
            targets = [{k: v.to(device) for k,v in t.items()} for t in targets]

            optimizer.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast():
                loss_dict = model(imgs, targets)
                loss = sum(loss_dict.values())
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            running += loss.item()

        scheduler.step()
        train_loss = running / max(1, len(train_loader))

        # ===== avaliação (mAP) =====
        stats, ev = eval_coco_shim(model, val_loader, score_thr=score_thr_eval)
        ap50 = float(stats[1]) if stats is not None and len(stats) >= 2 else 0.0

        print(f"[{epoch:02d}] train_loss={train_loss:.4f} | AP50(val)={ap50:.3f}")
        if ev is not None:
            per_cls = coco_ap50_per_class(ev)
            if per_cls:
                txt = " | ".join([f"{name}:{ap:.3f}" for _,name,ap in per_cls])
                print("  AP50 por classe:", txt)

        # ===== checkpoint pelo AP50 =====
        if ap50 > best_ap50:
            best_ap50 = ap50
            torch.save(model.state_dict(), ckpt_path)
            print(f"  ↳ ✅ melhor até agora (AP50={best_ap50:.3f}) salvo em {ckpt_path}")


In [10]:
# 2.1 – congele o backbone por 2-3 épocas para estabilizar
for p in model.backbone.body.parameters():
    p.requires_grad = False

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=5e-4)

# Cosine Annealing simples; alternativa: StepLR( step_size=8, gamma=0.1 )
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)
scaler = torch.cuda.amp.GradScaler()

/tmp/ipython-input-3059598682.py:10: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [18]:
fit(model, train_loader, val_loader, optimizer, scaler, scheduler, epochs=20, ckpt_path="/content/drive/MyDrive/TCC/checkpoints/fasterrcnn_combined_ds_ft_bb.pth")


/tmp/ipython-input-71931217.py:13: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


AttributeError: 'ConcatDataset' object has no attribute 'coco'

## Avaliação primeiro treinamento

IoU, matching e AP por classe

In [12]:
import os, math, pathlib
import numpy as np
import torch
import cv2

ID_TO_NAME = {1: "Piscina", 2: "CaixaDagua"}

def iou_matrix(boxes1, boxes2):
    # boxes: [N,4] e [M,4] no formato [x1,y1,x2,y2]
    if boxes1.numel() == 0 or boxes2.numel() == 0:
        return torch.zeros((boxes1.shape[0], boxes2.shape[0]))
    x11, y11, x12, y12 = boxes1[:,0], boxes1[:,1], boxes1[:,2], boxes1[:,3]
    x21, y21, x22, y22 = boxes2[:,0], boxes2[:,1], boxes2[:,2], boxes2[:,3]
    xa = torch.maximum(x11[:,None], x21[None,:])
    ya = torch.maximum(y11[:,None], y21[None,:])
    xb = torch.minimum(x12[:,None], x22[None,:])
    yb = torch.minimum(y12[:,None], y22[None,:])
    inter = (xb - xa).clamp(min=0) * (yb - ya).clamp(min=0)
    area1 = (x12 - x11).clamp(min=0) * (y12 - y11).clamp(min=0)
    area2 = (x22 - x21).clamp(min=0) * (y22 - y21).clamp(min=0)
    union = area1[:,None] + area2[None,:] - inter
    return inter / union.clamp(min=1e-9)

def average_precision(scores, tps, fps, npos):
    # Ordena por score desc, acumula TP/FP e integra PR
    if len(scores) == 0:
        return 0.0, {"precision":[], "recall":[], "scores":[]}
    order = np.argsort(-scores)
    tps = np.array(tps)[order].astype(np.float32)
    fps = np.array(fps)[order].astype(np.float32)

    tp_cum = np.cumsum(tps)
    fp_cum = np.cumsum(fps)
    recall = tp_cum / max(npos, 1)
    precision = tp_cum / np.maximum(tp_cum + fp_cum, 1e-9)

    # AP pela integração contínua (VOC moderno)
    # Força envelope de precisão não-crescente
    mrec = np.concatenate(([0.0], recall, [1.0]))
    mpre = np.concatenate(([0.0], precision, [0.0]))
    for i in range(mpre.size - 1, 0, -1):
        mpre[i-1] = max(mpre[i-1], mpre[i])
    # Pega pontos onde recall muda
    idx = np.where(mrec[1:] != mrec[:-1])[0]
    ap = float(np.sum((mrec[idx + 1] - mrec[idx]) * mpre[idx + 1]))
    return ap, {"precision":precision.tolist(), "recall":recall.tolist(), "scores":scores[order].tolist()}


Avaliação por classe (AP@0.50) e salvamento de imagens com boxes

In [13]:
@torch.no_grad()
def evaluate_and_visualize(model, loader, out_dir="/content/out", score_thr=0.25, iou_thr=0.50, max_images_to_save=200):
    model.eval()
    pathlib.Path(out_dir).mkdir(parents=True, exist_ok=True)

    # Acumuladores por classe
    classes = [1,2]
    scores_by_c = {c: [] for c in classes}
    tps_by_c    = {c: [] for c in classes}
    fps_by_c    = {c: [] for c in classes}
    npos_by_c   = {c: 0  for c in classes}

    saved = 0

    for imgs, targets in loader:
        # move
        imgs_cuda = [im.to(device) for im in imgs]
        preds = model(imgs_cuda)
        preds = [{k: v.detach().cpu() for k,v in p.items()} for p in preds]
        tgts  = [{k: v.detach().cpu() for k,v in t.items()} for t in targets]

        for img_tensor, pred, tgt in zip(imgs, preds, tgts):
            # Contagem de GT por classe
            gt_boxes = tgt["boxes"]
            gt_labels = tgt["labels"]
            for c in classes:
                npos_by_c[c] += int((gt_labels==c).sum().item())

            # Filtra preds por score
            keep = pred["scores"] >= score_thr
            pb = pred["boxes"][keep]
            pl = pred["labels"][keep]
            ps = pred["scores"][keep]

            # Matching por classe
            for c in classes:
                pb_c = pb[pl==c]
                ps_c = ps[pl==c]
                gt_c = gt_boxes[gt_labels==c]

                if len(pb_c)==0:
                    continue
                # Greedy: ordena por score, faz matching one-to-one por IoU
                order = torch.argsort(ps_c, descending=True)
                pb_c = pb_c[order]
                ps_c = ps_c[order]

                matched_gt = torch.zeros(len(gt_c), dtype=torch.bool)
                for j, box in enumerate(pb_c):
                    if len(gt_c)==0:
                        # nenhuma GT daquela classe nessa imagem
                        scores_by_c[c].append(float(ps_c[j]))
                        tps_by_c[c].append(0)
                        fps_by_c[c].append(1)
                        continue
                    ious = iou_matrix(box.unsqueeze(0), gt_c).squeeze(0)  # [num_gt]
                    best_iou, best_idx = (ious.max().item(), int(ious.argmax().item()))
                    scores_by_c[c].append(float(ps_c[j]))
                    if best_iou >= iou_thr and not matched_gt[best_idx]:
                        tps_by_c[c].append(1); fps_by_c[c].append(0)
                        matched_gt[best_idx] = True
                    else:
                        tps_by_c[c].append(0); fps_by_c[c].append(1)

            # ====== Visualização / salvamento ======
            if saved < max_images_to_save:
                # Converte imagem tensor -> BGR uint8
                im = img_tensor.detach().cpu()
                if im.dtype==torch.float32:
                    # assume [0,1]
                    im = (im*255).clamp(0,255).byte()
                im = im.permute(1,2,0).numpy()  # HWC, RGB
                im = cv2.cvtColor(im, cv2.COLOR_RGB2BGR)

                # Desenha GT (tracejado fino)
                for b, lab in zip(gt_boxes, gt_labels):
                    x1,y1,x2,y2 = map(int, b.tolist())
                    color = (0,255,0) if lab.item()==1 else (255,0,0)
                    cv2.rectangle(im, (x1,y1), (x2,y2), color, 1, lineType=cv2.LINE_AA)

                # Desenha Preds
                for b, lab, sc in zip(pb, pl, ps):
                    x1,y1,x2,y2 = map(int, b.tolist())
                    color = (50,220,50) if lab.item()==1 else (50,50,220)
                    cv2.rectangle(im, (x1,y1), (x2,y2), color, 2, lineType=cv2.LINE_AA)
                    txt = f"{ID_TO_NAME.get(int(lab.item()), str(int(lab.item())))} {float(sc):.2f}"
                    cv2.putText(im, txt, (x1, max(y1-5, 0)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2, cv2.LINE_AA)

                out_path = os.path.join(out_dir, f"pred_{saved:05d}.jpg")
                cv2.imwrite(out_path, im)
                saved += 1

    # Calcula AP@0.50 por classe
    results = {}
    for c in classes:
        ap, curves = average_precision(
            np.array(scores_by_c[c], dtype=np.float32),
            tps_by_c[c],
            fps_by_c[c],
            npos_by_c[c]
        )
        results[c] = {"AP@0.50": ap, "npos": int(npos_by_c[c])}

    return results


Armazena imagens com boxes nas piscinas e caixas d'agua e salva checkpoint

In [14]:
out = evaluate_and_visualize(model, test_loader, out_dir="/content/drive/MyDrive/TCC/out", score_thr=0.25, iou_thr=0.50, max_images_to_save=10)

print("==== Resultados por classe (AP@0.50) ====")
for cid, info in out.items():
    print(f"{ID_TO_NAME[cid]:<12}  AP@0.50 = {info['AP@0.50']:.3f}   (GT objs: {info['npos']})")

print("Imagens salvas em /content/out (preds em cor forte; GT em traço fino).")


==== Resultados por classe (AP@0.50) ====
Piscina       AP@0.50 = 0.743   (GT objs: 1109)
CaixaDagua    AP@0.50 = 0.004   (GT objs: 2723)
Imagens salvas em /content/out (preds em cor forte; GT em traço fino).


## Inicia segunda rodade de treinamentos com foco em caixas dagua

In [27]:
import random, numpy as np, torch
random.seed(42); np.random.seed(42); torch.manual_seed(42); torch.cuda.manual_seed_all(42)
torch.backends.cudnn.benchmark = True
# opcional (se quiser estabilidade extra):
# !pip install -q "torch==2.4.0" "torchvision==0.19.0"


In [31]:
import numpy as np
from torch.utils.data import WeightedRandomSampler, DataLoader, ConcatDataset
def has_cls2(ds):
    flags=[]
    for i in range(len(ds)):
        _,t=ds[i]; flags.append(2 in t["labels"].tolist())
    return np.array(flags, bool)
flags = has_cls2(train_ds)
weights = np.where(flags, 4.0, 1.0).astype(np.float32)  # 4x mais chance de vir img com caixa
sampler = WeightedRandomSampler(weights, num_samples=len(weights), replacement=True)

train_loader = DataLoader(train_ds, batch_size=2, sampler=sampler, num_workers=2, pin_memory=True, collate_fn=collate_fn)
val_loader   = DataLoader(val_ds,   batch_size=2, shuffle=False,   num_workers=2, pin_memory=True, collate_fn=collate_fn)
test_loader  = DataLoader(test_ds,  batch_size=2, shuffle=False,   num_workers=2, pin_memory=True, collate_fn=collate_fn)


In [30]:
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ckpt_path = "/content/drive/MyDrive/TCC/checkpoints/fasterrcnn_combined_ds_ft.pth"  # seu arquivo

# 1) Recrie a MESMA arquitetura do treino anterior (v2) SEM pesos COCO
model = fasterrcnn_resnet50_fpn_v2(weights=None)

# 2) Ajuste o head para o mesmo nº de classes usado no seu projeto
NUM_CLASSES = 3  # 0=bg, 1=Piscina, 2=CaixaDagua
in_f = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_f, NUM_CLASSES)

# 3) Carregue o checkpoint de forma robusta (parcial se shapes não baterem)
state = torch.load(ckpt_path, map_location="cpu")
cur = model.state_dict()
filtered = {k:v for k,v in state.items() if k in cur and cur[k].shape == v.shape}
cur.update(filtered)
missing, unexpected = model.load_state_dict(cur, strict=False)
print("Aproveitados:", len(filtered), "| missing:", len(missing), "| unexpected:", len(unexpected))

model.to(device).eval()

# 4) Sanidade rápida: head coerente e um forward “seco”
cs = model.roi_heads.box_predictor.cls_score
bp = model.roi_heads.box_predictor.bbox_pred
print("cls_out, bbox_out =", cs.out_features, bp.out_features)  # deve ser 3 e 12

imgs, _ = next(iter(val_loader))  # use seu val_loader
with torch.no_grad():
    _ = model([im.to(device) for im in imgs])  # deve passar sem erro


Aproveitados: 404 | missing: 0 | unexpected: 0
cls_out, bbox_out = 3 12


In [32]:
# próximos do default; depois abrimos o funil se precisar
model.rpn.pre_nms_top_n_train  = 2000
model.rpn.post_nms_top_n_train = 1000
model.rpn.pre_nms_top_n_test   = 1000
model.rpn.post_nms_top_n_test  = 1000
model.roi_heads.batch_size_per_image = 256
model.roi_heads.positive_fraction    = 0.25

In [35]:
import contextlib, torch
USE_CUDA = torch.cuda.is_available()
amp_ctx = (torch.amp.autocast('cuda') if USE_CUDA else contextlib.nullcontext())

@torch.no_grad()
def eval_ap_c2(model, val_loader):
    # sua evaluate_and_visualize deve aceitar out_dir=None e max_images_to_save=0
    res = evaluate_and_visualize(model, val_loader, out_dir="/content/drive/MyDrive/TCC/out", score_thr=0.25, iou_thr=0.50, max_images_to_save=0)
    return res.get(2, {}).get("AP@0.50", 0.0)

def train_epochs(model, train_loader, val_loader, opt, sched, epochs, ckpt_path):
    best = -1.0
    dev = next(model.parameters()).device
    for ep in range(1, epochs+1):
        model.train(); run=0.0
        for imgs, tgts in train_loader:
            imgs=[im.to(dev) for im in imgs]
            tgts=[{k:v.to(dev) for k,v in t.items()} for t in tgts]
            opt.zero_grad(set_to_none=True)
            with amp_ctx:
                loss = sum(model(imgs, tgts).values())
            loss.backward(); opt.step()
            run += loss.item()
        sched.step()
        model.eval()
        ap_c2 = eval_ap_c2(model, val_loader)
        print(f"[{ep:02d}] loss={run/len(train_loader):.4f} | AP50(Caixa)={ap_c2:.3f}")
        if ap_c2 > best:
            best = ap_c2
            torch.save(model.state_dict(), ckpt_path)
            print(f"  ↑ melhor AP50(Caixa) {best:.3f} salvo em {ckpt_path}")
    return best

# Estágio A — congelar backbone (3 épocas rápidas)
for p in model.backbone.body.parameters(): p.requires_grad = False
optA  = torch.optim.SGD([p for p in model.parameters() if p.requires_grad], lr=2e-3, momentum=0.9, weight_decay=5e-4)
schA  = torch.optim.lr_scheduler.MultiStepLR(optA, milestones=[3], gamma=0.1)
bestA = train_epochs(model, train_loader, val_loader, optA, schA, epochs=3, ckpt_path="/content/drive/MyDrive/TCC/checkpoints/best_c2_heads.pth")

# Estágio B — liberar backbone (refino 10–12 épocas, LR menor)
for p in model.backbone.body.parameters(): p.requires_grad = True
optB  = torch.optim.SGD([p for p in model.parameters() if p.requires_grad], lr=8e-4, momentum=0.9, weight_decay=5e-4)
schB  = torch.optim.lr_scheduler.MultiStepLR(optB, milestones=[8,12], gamma=0.5)
bestB = train_epochs(model, train_loader, val_loader, optB, schB, epochs=12, ckpt_path="/content/drive/MyDrive/TCC/checkpoints/best_c2_full.pth")


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7b5b58b94fe0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1664, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1647, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7b5b58b94fe0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1664, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 16

[01] loss=0.6114 | AP50(Caixa)=0.101
  ↑ melhor AP50(Caixa) 0.101 salvo em /content/drive/MyDrive/TCC/checkpoints/best_c2_heads.pth
[02] loss=0.6036 | AP50(Caixa)=0.070
[03] loss=0.6092 | AP50(Caixa)=0.054
[01] loss=0.5615 | AP50(Caixa)=0.036
  ↑ melhor AP50(Caixa) 0.036 salvo em /content/drive/MyDrive/TCC/checkpoints/best_c2_full.pth
[02] loss=0.5573 | AP50(Caixa)=0.043
  ↑ melhor AP50(Caixa) 0.043 salvo em /content/drive/MyDrive/TCC/checkpoints/best_c2_full.pth
[03] loss=0.5669 | AP50(Caixa)=0.099
  ↑ melhor AP50(Caixa) 0.099 salvo em /content/drive/MyDrive/TCC/checkpoints/best_c2_full.pth
[04] loss=0.5435 | AP50(Caixa)=0.029
[05] loss=0.5473 | AP50(Caixa)=0.019
[06] loss=0.5468 | AP50(Caixa)=0.021
[07] loss=0.5500 | AP50(Caixa)=0.044
[08] loss=0.5445 | AP50(Caixa)=0.045
[09] loss=0.5342 | AP50(Caixa)=0.022
[10] loss=0.5357 | AP50(Caixa)=0.072
[11] loss=0.5378 | AP50(Caixa)=0.034
[12] loss=0.5340 | AP50(Caixa)=0.046


In [36]:
model.rpn.pre_nms_top_n_train  = 4000
model.rpn.post_nms_top_n_train = 2000
model.rpn.pre_nms_top_n_test   = 2000
model.rpn.post_nms_top_n_test  = 1500
model.roi_heads.batch_size_per_image = 512
model.roi_heads.positive_fraction    = 0.5


In [37]:
model.load_state_dict(torch.load("/content/drive/MyDrive/TCC/checkpoints/best_c2_full.pth", map_location=next(model.parameters()).device))
model.eval()
res_test = evaluate_and_visualize(model, test_loader, out_dir="/content/out_v2", score_thr=0.25, iou_thr=0.50, max_images_to_save=200)
print("TEST AP50:", {"Piscina":res_test[1]["AP@0.50"], "Caixa":res_test[2]["AP@0.50"]})
print("Imagens em /content/out_v2")


TEST AP50: {'Piscina': 0.5377687034308409, 'Caixa': 0.04010817213592527}
Imagens em /content/out_v2


## Adiciona camada de attention

Class AttnWoMLPHead

In [46]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AttnTwoMLPHead(nn.Module):
    """
    Head para Faster R-CNN com atenção nas features de cada RoI.
    Espera tensores N x C x H x W (normalmente H=W=7, C=256).
    Pipeline:
      [N, C, 7, 7] --(flatten spatial->tokens)--> [N, 49, C]
      --(LayerNorm + MultiheadAttention)--> [N, 49, C]
      --(mean pool em tokens)--> [N, C]
      --(MLP 2 camadas)--> [N, representation_size]
    """
    def __init__(self, in_channels: int, resolution: int = 7,
                 representation_size: int = 1024, num_heads: int = 8,
                 attn_dropout: float = 0.0, mlp_dropout: float = 0.1):
        super().__init__()
        self.in_channels = in_channels
        self.resolution  = resolution
        self.num_tokens  = resolution * resolution
        self.embed_dim   = in_channels
        self.representation_size = representation_size

        self.ln1 = nn.LayerNorm(self.embed_dim)
        self.attn = nn.MultiheadAttention(
            embed_dim=self.embed_dim,
            num_heads=num_heads,
            dropout=attn_dropout,
            batch_first=True,   # usa [N, S, E]
        )
        self.ln2 = nn.LayerNorm(self.embed_dim)

        # MLP igual ao TwoMLPHead (duas FCs) depois do pooling dos tokens
        self.fc6 = nn.Linear(self.embed_dim, representation_size)
        self.fc7 = nn.Linear(representation_size, representation_size)
        self.drop = nn.Dropout(mlp_dropout)
        self.out_features = representation_size  # ajuda na recomposição do predictor

        # inicializações estáveis
        nn.init.xavier_uniform_(self.fc6.weight); nn.init.zeros_(self.fc6.bias)
        nn.init.xavier_uniform_(self.fc7.weight); nn.init.zeros_(self.fc7.bias)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: [N, C, H, W]  (H=W=self.resolution)
        N, C, H, W = x.shape
        assert C == self.in_channels and H == self.resolution and W == self.resolution, \
            f"shape inesperada: {x.shape}, esperado: [N,{self.in_channels},{self.resolution},{self.resolution}]"

        # 1) vira tokens: [N, S=H*W, C]
        x = x.reshape(N, C, H * W).transpose(1, 2)  # [N, S, C]

        # 2) MHSA (pré-norm) + residual
        h = self.ln1(x)
        attn_out, _ = self.attn(h, h, h, need_weights=False)  # [N, S, C]
        x = x + attn_out

        # 3) pós-attn norm (leve estabilização)
        x = self.ln2(x)

        # 4) pool sobre tokens (mean) -> [N, C]
        x = x.mean(dim=1)

        # 5) MLP 2 camadas
        x = F.relu(self.fc6(x))
        x = self.drop(x)
        x = F.relu(self.fc7(x))
        return x

In [47]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1) crie seu modelo v2 (se quiser, pode usar pesos COCO e depois trocar head)
model = fasterrcnn_resnet50_fpn_v2(weights=None)  # ou use COCO_V1 se preferir warm-start no backbone

# 2) substitui a box_head pelo head com atenção
roi_pool_res = model.roi_heads.box_roi_pool.output_size[0]   # normalmente 7
C_backbone   = model.backbone.out_channels                    # normalmente 256

attn_head = AttnTwoMLPHead(
    in_channels=C_backbone,
    resolution=roi_pool_res,
    representation_size=1024,  # mantenha 1024 para compat com predictor padrão
    num_heads=8,               # você pode testar 4/8/16
    attn_dropout=0.0,
    mlp_dropout=0.1,
)
model.roi_heads.box_head = attn_head

# 3) reconstroi o predictor para 3 classes (bg + piscina + caixa)
in_features = attn_head.out_features  # 1024
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 3)

model.to(device)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       

In [48]:
state = torch.load("/content/drive/MyDrive/TCC/checkpoints/fasterrcnn_combined_ds_ft.pth", map_location="cpu")
cur = model.state_dict()
filtered = {k:v for k,v in state.items() if k in cur and cur[k].shape == v.shape}
cur.update(filtered)
model.load_state_dict(cur, strict=False)


<All keys matched successfully>

In [75]:
import torch
import torch.nn as nn

class CBAMv2(nn.Module):
    version = "cbam_v2_safe_max"  # tag p/ debug

    def __init__(self, channels, reduction=16, kernel_size=7):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(channels, channels // reduction),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels),
        )
        self.conv_spatial = nn.Conv2d(2, 1, kernel_size, padding=kernel_size // 2, bias=False)

    def forward(self, x):
        # x: [N, C, H, W]
        N, C, H, W = x.shape

        # Channel attention (usar max em 2 etapas)
        avg_pool = torch.mean(x, dim=(2, 3), keepdim=True)                     # [N,C,1,1]
        max_pool = torch.max(torch.max(x, dim=2, keepdim=True)[0], dim=3, keepdim=True)[0]  # [N,C,1,1]

        avg_w = self.mlp(avg_pool.flatten(1))                                  # [N,C]
        max_w = self.mlp(max_pool.flatten(1))                                  # [N,C]
        ch_attn = torch.sigmoid(avg_w + max_w).view(N, C, 1, 1)                # [N,C,1,1]
        x = x * ch_attn

        # Spatial attention (usar max 1D)
        avg_map = torch.mean(x, dim=1, keepdim=True)                           # [N,1,H,W]
        max_map = torch.max(x, dim=1, keepdim=True)[0]                         # [N,1,H,W]
        s = torch.cat([avg_map, max_map], dim=1)                               # [N,2,H,W]
        sp_attn = torch.sigmoid(self.conv_spatial(s))                          # [N,1,H,W]
        return x * sp_attn


In [94]:
class FPNWithCBAMv4(nn.Module):
    version = "fpn_with_cbam_v4_ignore_pool"
    def __init__(self, fpn_core):
        super().__init__()
        self.fpn = fpn_core
        self.cbam = nn.ModuleDict()  # criaremos por nível quando necessário

    def forward(self, x):
        feats = self.fpn(x)   # OrderedDict, e.g. ['0','1','2','3','pool']
        out = {}
        for k, v in feats.items():
            if isinstance(k, str) and k == "pool":
                out[k] = v
                continue
            key = str(k)
            if key not in self.cbam:
                m = CBAMv2(v.shape[1])
                m.to(v.device)                      # <<< garante device certo
                self.cbam[key] = m
            else:
                # se o módulo existir mas estiver em outro device, mova-o
                mod = self.cbam[key]
                if next(mod.parameters()).device != v.device:
                    mod.to(v.device)                # <<< realoca para o device do tensor
            out[k] = self.cbam[key](v)
        return out


In [95]:
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

device = next(model.parameters()).device if 'model' in globals() else ('cuda' if torch.cuda.is_available() else 'cpu')
if isinstance(device, str):
    device = torch.device(device)

# (Re)crie o modelo base se preferir limpar estado anterior, ou apenas reconstrua a FPN
# model = fasterrcnn_resnet50_fpn_v2(weights=None).to(device)

# pegue o núcleo da FPN e re-encape com v4
fpn_core = model.backbone.fpn.fpn if hasattr(model.backbone.fpn, "fpn") else model.backbone.fpn
model.backbone.fpn = FPNWithCBAMv4(fpn_core)

# reforce o head (3 classes) caso tenha refeito o modelo
in_f = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_f, 3)
model.to(device)

print("FPN wrapper:", getattr(model.backbone.fpn, "version", None))
print("CBAM submodules:", list(model.backbone.fpn.cbam.keys()))


FPN wrapper: fpn_with_cbam_v4_ignore_pool
CBAM submodules: []


In [96]:
model.eval()
imgs, _ = next(iter(val_loader))
with torch.no_grad():
    _ = model([im.to(device) for im in imgs])
print("✅ Forward eval passou (CBAM v4, pool ignorado)")


✅ Forward eval passou (CBAM v4, pool ignorado)


In [97]:
# Estágio A — backbone congelado
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for p in model.backbone.body.parameters(): p.requires_grad = False
optA = torch.optim.SGD([p for p in model.parameters() if p.requires_grad], lr=8e-4, momentum=0.9, weight_decay=5e-4)
schA = torch.optim.lr_scheduler.MultiStepLR(optA, milestones=[3], gamma=0.1)
bestA = train_epochs(model, train_loader, val_loader, optA, schA, epochs=3,
                     ckpt_path="/content/drive/MyDrive/TCC/checkpoints/attn_heads.pth")

# Estágio B — libera backbone
for p in model.backbone.body.parameters(): p.requires_grad = True
optB = torch.optim.SGD([p for p in model.parameters() if p.requires_grad], lr=6e-4, momentum=0.9, weight_decay=5e-4)
schB = torch.optim.lr_scheduler.MultiStepLR(optB, milestones=[8,12], gamma=0.5)
bestB = train_epochs(model, train_loader, val_loader, optB, schB, epochs=10,
                     ckpt_path="/content/drive/MyDrive/TCC/checkpoints/attn_full.pth")


[01] loss=2.3442 | AP50(Caixa)=0.000
  ↑ melhor AP50(Caixa) 0.000 salvo em /content/drive/MyDrive/TCC/checkpoints/attn_heads.pth
[02] loss=1.4639 | AP50(Caixa)=0.000
[03] loss=1.3004 | AP50(Caixa)=0.000
[01] loss=1.2724 | AP50(Caixa)=0.000
  ↑ melhor AP50(Caixa) 0.000 salvo em /content/drive/MyDrive/TCC/checkpoints/attn_full.pth
[02] loss=1.2582 | AP50(Caixa)=0.000
[03] loss=1.2382 | AP50(Caixa)=0.000
[04] loss=1.1858 | AP50(Caixa)=0.000
[05] loss=1.0967 | AP50(Caixa)=0.000


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7b5b58b94fe0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1664, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1628, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.12/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/connection.py", line 1136, in wait
    ready = selector.select(timeout)
            ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/selectors.py", line 415, in select
    fd_event_list = self._selector.poll(timeout)
    

KeyboardInterrupt: 